In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets

from pandas_datareader import data as pdr

import pandas as pd

import yfinance as yf
yf.pdr_override()
from datetime import datetime as calendar
import numpy as np
import chart_studio.plotly as plty

secrets = ReadSecrets().read('../secrets.json')
plty.sign_in(secrets.secrets['plotly']['username'], secrets.secrets['plotly']['apiKey'])

import plotly.graph_objs as gobjs

/home/andre/Desktop/pythonTradingAlgos/tradingalgos/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def get_history(tickers, start_date, end_date):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=start_date, end=end_date))
        
    tickers_data = map(data, tickers)
    
    return pd.concat(tickers_data, keys=tickers, names = ['Ticker', 'Date'])

In [14]:
tickers = ['AAPL', 'MSFT', '^GSPC']
start_date = calendar(2016, 1 ,1)
end_date = calendar(2020, 2, 1)

data = get_history(tickers, start_date, end_date)
data = data.reset_index()
data = data.set_index(['Date', 'Ticker']).sort_index()
close = data['Close']

weekdays = pd.date_range(start=start_date, end=end_date, freq='B') # all weekdays in the interval
close = close.reindex(pd.MultiIndex.from_product([weekdays, tickers], names=['Date', 'Ticker']), fill_value=np.NaN)

close = close.reset_index().pivot(index='Date', columns='Ticker', values='Close')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [15]:
msft = data.xs('MSFT', level='Ticker', drop_level=False).dropna()
display(msft)

,,Open,High,Low,Close,Adj Close,Volume
Date,Ticker,,,,,,
2016-01-04,MSFT,54.320000,54.799999,53.389999,54.799999,50.536495,53778000
2016-01-05,MSFT,54.930000,55.389999,54.540001,55.049999,50.767044,34079700
2016-01-06,MSFT,54.320000,54.400002,53.639999,54.049999,49.844841,39518900
2016-01-07,MSFT,52.700001,53.490002,52.070000,52.169998,48.111115,56564900
2016-01-08,MSFT,52.369999,53.279999,52.150002,52.330002,48.258663,48754000
...,...,...,...,...,...,...,...
2020-01-27,MSFT,161.149994,163.380005,160.199997,162.279999,162.279999,32078100
2020-01-28,MSFT,163.779999,165.759995,163.070007,165.460007,165.460007,24899900
2020-01-29,MSFT,167.839996,168.750000,165.690002,168.039993,168.039993,34754500


In [16]:
window_days = 21
moving_average = msft.Close.rolling(window=window_days).mean()
#msft = msft['2018-06-01':]

In [17]:
display(msft)

,,Open,High,Low,Close,Adj Close,Volume
Date,Ticker,,,,,,
2016-01-04,MSFT,54.320000,54.799999,53.389999,54.799999,50.536495,53778000
2016-01-05,MSFT,54.930000,55.389999,54.540001,55.049999,50.767044,34079700
2016-01-06,MSFT,54.320000,54.400002,53.639999,54.049999,49.844841,39518900
2016-01-07,MSFT,52.700001,53.490002,52.070000,52.169998,48.111115,56564900
2016-01-08,MSFT,52.369999,53.279999,52.150002,52.330002,48.258663,48754000
...,...,...,...,...,...,...,...
2020-01-27,MSFT,161.149994,163.380005,160.199997,162.279999,162.279999,32078100
2020-01-28,MSFT,163.779999,165.759995,163.070007,165.460007,165.460007,24899900
2020-01-29,MSFT,167.839996,168.750000,165.690002,168.039993,168.039993,34754500


In [18]:
msft = msft.reset_index(level=1, drop=True)
moving_average = moving_average.reset_index(level=1, drop=True)
display(msft)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,54.320000,54.799999,53.389999,54.799999,50.536495,53778000
2016-01-05,54.930000,55.389999,54.540001,55.049999,50.767044,34079700
2016-01-06,54.320000,54.400002,53.639999,54.049999,49.844841,39518900
2016-01-07,52.700001,53.490002,52.070000,52.169998,48.111115,56564900
2016-01-08,52.369999,53.279999,52.150002,52.330002,48.258663,48754000
...,...,...,...,...,...,...
2020-01-27,161.149994,163.380005,160.199997,162.279999,162.279999,32078100
2020-01-28,163.779999,165.759995,163.070007,165.460007,165.460007,24899900
2020-01-29,167.839996,168.750000,165.690002,168.039993,168.039993,34754500


In [19]:
trace_avg = gobjs.Scatter(
                x=moving_average.index,
                y=moving_average,
                name = f"MSFT MA({window_days})",
                line = dict(color = '#BEBECF'),
                opacity = 1)

trace_candles = gobjs.Candlestick(x=msft.index,
                    open=msft.Open,
                    high=msft.High,
                    low=msft.Low,
                    close=msft.Close,
                    name = "Price")


fig = dict(data=[trace_candles, trace_avg])
plty.iplot(fig)